In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/algorithms')
# from ce_utils.data import load_image_from_path, gen_label
from ce_utils.data import cv_test_data_load
from ce_model.cnn_inf import cnn_model
from ce_utils.cls_eval import cls_metric

# 1. Evaluation

In [2]:
model_list = ['no_sample_mixed', 'no_sample_equal', 'undersample', 'oversample']
model_dir = 'model/'
gpu_idx = 3

model_test = {}

df = pd.DataFrame(columns = ['model', 'Accr', 'Sens', 'Spec', 'NPV', 'PPV'])

for m, model_name in enumerate(model_list):
    print(model_name)
    model_test[model_name] = []
    for i in range(1, 6):
        print('\n{:02d}_fold'.format(i))
        np_test_name, np_test_X, np_test_Y = cv_test_data_load(filename = 'data_config_p3_2_negative_np-hd_---_--_5f_cv.pkl', 
                                                    n_fold=i)
        model = cnn_model(network = 'CNN_v1', n_ch = 3, n_cls = 2, 
                  model_dir = model_dir, model_name = '{:02d}_np-hd_p32_neg_{}'.format(i, model_name), 
                  gpu_idx = gpu_idx)
        
        model.inference(np_test_X, batch_size = 20)
        model_test[model_name].append(cls_metric(np_test_Y, model.score).cm2metric())
    print('')
    
    df.loc[m] = [model_name] + list(np.mean(model_test[model_name], axis = 0))
#     if i != 0:
#         break
df.to_csv('model_performance.csv',encoding='utf-8-sig', index = False)

no_sample_mixed

01_fold
class 0 : 1561
class 1 : 626
model:
01_np-hd_p32_neg_no_sample_mixed_0.0001_32_2010271441_195_t_accr_0.9799_t_loss_0.332701_v_accr_0.9583_v_loss_0.354452.pt

2187: 110(*20) |################################################## | 100.0% prediction

02_fold
class 0 : 1561
class 1 : 626
model:
02_np-hd_p32_neg_no_sample_mixed_0.0001_32_2010271818_009_t_accr_0.7142_t_loss_0.599078_v_accr_0.7137_v_loss_0.599547.pt

2187: 110(*20) |################################################## | 100.0% prediction
Totally misclassified class exists

03_fold
class 0 : 1561


/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/algorithms/ce_utils/cls_eval.py:48: RuntimeWarning: invalid value encountered in long_scalars
  ppv = round(100*self.cm[1, 1] / np.sum(self.cm[:, 1]), 2)


class 1 : 626
model:
03_np-hd_p32_neg_no_sample_mixed_0.0001_32_2009100511_312_t_accr_0.9876_t_loss_0.325609_v_accr_0.9509_v_loss_0.362822.pt

2187: 110(*20) |################################################## | 100.0% prediction

04_fold
class 0 : 1561
class 1 : 626
model:
04_np-hd_p32_neg_no_sample_mixed_0.0001_32_2009101515_360_t_accr_0.9932_t_loss_0.320155_v_accr_0.9566_v_loss_0.355224.pt

2187: 110(*20) |################################################## | 100.0% prediction

05_fold
class 0 : 1560
class 1 : 625
model:
05_np-hd_p32_neg_no_sample_mixed_0.0001_32_2010280318_290_t_accr_0.9864_t_loss_0.326850_v_accr_0.9480_v_loss_0.364498.pt

2185: 110(*20) |################################################## | 100.0% prediction

no_sample_equal

01_fold
class 0 : 1561
class 1 : 626
model:
01_np-hd_p32_neg_no_sample_equal_0.0001_16_2010161827_372_t_accr_0.9913_t_loss_0.321792_v_accr_0.9543_v_loss_0.358867.pt

2187: 110(*20) |################################################## | 100.0% pr

In [3]:
df

,model,Accr,Sens,Spec,NPV,PPV
0,no_sample_mixed,85.540,52.364,98.834,85.590,56.846
1,no_sample_equal,95.288,90.094,97.372,96.086,93.246
2,undersample,94.924,91.112,96.450,96.438,91.164
3,oversample,94.640,89.390,96.746,95.788,91.728


In [3]:
df

,model,Accr,Sens,Spec,NPV,PPV
0,no_sample_mixed,90.170,70.028,98.244,90.390,75.288
1,no_sample_equal,95.288,90.094,97.372,96.086,93.246
2,undersample,94.924,91.112,96.450,96.438,91.164
3,oversample,94.640,89.390,96.746,95.788,91.728


# 2. Discussion

- 각 fold마다 network의 parameter가 동일하게 시작할 수 있도록 reproducible하게 훈련했는데, no_sample case에서 훈련이 안 되거나 덜 된 케이스가 보였음
- 예상대로 sampling 안 쓰고 통으로 (mixed) batch를 뽑은 훈련방식은 major class ('negative')에 모델이 bias되는 경향이 확실하게 크다는 것을 알 수 있음
- no sample (equal)이 undersample 및 oversample과 비등비등하길 원했는데...
- 0.0001, 32 이외에 learning rate를 바꿔가면서 훈련시켜볼 필요가 있음..
